Imports

In [1]:
import sys
import json
from pprint import pprint
import csv
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk

Dataset Read

In [42]:
def read_csv(file_name):
    df = pd.read_csv(file_name)
    return df

In [53]:
houses = read_csv('housing.csv')
houses.head()

,City,Size
0,LA,800
1,Tehran,1000
2,NY,1100


In [54]:
predicted_prices = read_csv('predicted_prices.csv')
predicted_prices.head()

,Size,Predicted Price
0,800,8500
1,1000,9000
2,1100,9200


Left Join

In [55]:
joined_csvs = pd.merge(houses, predicted_prices, on='Size', how='left')
joined_csvs.head()

,City,Size,Predicted Price
0,LA,800,8500
1,Tehran,1000,9000
2,NY,1100,9200


Elastic Search Connection

In [60]:
es = Elasticsearch(
    ['localhost'],
    port=9200
)

es.ping()

True

In [74]:
housing_index = 'housing_test'

### Adding Data to ElasticSearch

Add - Not bulky:

In [69]:
def add_not_bulky(df, index_name):
    for _, row in df.iterrows():
        document = row.to_dict()
        es.index(index=index_name, body=document, doc_type=csv)

Add - Bulky:

In [75]:
def generator(df, index_name):
    for _, row in df.iterrows():
        yield {
            '_index': index_name,
            '_type': "_doc",
            '_source': row.to_json()
        }

In [76]:
bulk(es, generator(houses, housing_index))

(3, [])

API Call

In [77]:
search_query = {
    "query": {
        "match_all": {}
    }
}

response = es.transport.perform_request(
    method='GET',
    url='/' + housing_index + '/_search',
    body=search_query
)

hits = response['hits']['hits']
for hit in hits:
    source = hit['_source']
    print(source)

{'City': 'LA', 'Size': 800}
{'City': 'Tehran', 'Size': 1000}
{'City': 'NY', 'Size': 1100}


Reading Data From ElasticSearch

In [78]:
result = es.search(index=housing_index, body=search_query, size=1000, )

recieved_from_es = pd.DataFrame([hit['_source'] for hit in result['hits']['hits']])

recieved_from_es.head()

,City,Size
0,LA,800
1,Tehran,1000
2,NY,1100
